In [2]:
import pandas as pd
import re
from tqdm.notebook import tqdm
import time
from datetime import datetime

In [3]:
df = pd.read_csv("reestr_17_1831.csv",encoding='utf-8',sep=';')

In [5]:
df.columns

Index(['населенный_пункт', 'тип_тс', 'тип_маршрута', 'номер_маршрута',
       'наим_маршрута', 'маршут_след_прям', 'маршут_след_обрат',
       'список_оп_прям', 'список_оп_обрат', 'режим_работы', 'интервалы',
       'выпуск', 'время_в_пути', 'протяженность', 'предприятие', 'подв_состав',
       'стоимость_проезда', 'место_отстоя', 'место_отправл', 'время_отправл',
       'интерв_ср_буд', 'интерв_ср_вых', 'интерв_ср_вчр', 'пнд', 'втр', 'срд',
       'чтв', 'птн', 'сбт', 'вск', 'ссылка'],
      dtype='object')

In [123]:
lst1 = list(df.выпуск.unique())

## Режим работы

In [55]:
def fnd_tm_strt_end(string):
    
    lst_dgt = re.findall(r'\d+', string)
    if len(lst_dgt) > 1:
        
        l1 = []
        for dgt in lst_dgt:
            l1.append(int(dgt))
        #
        l1.sort()
        tm_strt = min(l1)
        tm_end = max(l1)
        cnt=0
        while ((tm_strt < 5) & (tm_strt < tm_end)):
            try:
                tm_strt = min(l1[cnt:])
            except:
                tm_strt = 6
                break
            cnt+=1
        #
        cnt=len(l1)
        while ((tm_end > 23) & (tm_strt < tm_end)):
            try:
                tm_end = max(l1[:cnt])
            except:
                tm_end = 22
                break
            cnt-=1
        #
    #
    else:
        tm_strt = 6
        tm_end = 22
    #
    return tm_strt, tm_end

In [116]:
lst_reg_rab = list(df.режим_работы)

lst_strt_wrk,lst_end_wrk,lst_strt_hld,lst_end_hld = [],[],[],[]

i=0
for i in range(len(lst_reg_rab)):
    string = str(lst_reg_rab[i])
    if string == 'nan':
        tm_strt_wrk, tm_end_wrk = '-','-'
        tm_strt_hld, tm_end_hld = '-','-'
    #
    elif 'жедневно' in string:
        tm_strt_wrk, tm_end_wrk = fnd_tm_strt_end(string)
        tm_strt_hld, tm_end_hld = tm_strt_wrk, tm_end_wrk
    #
    elif 'рабочи' in string:
        ind_hld = string.find('выходны')
        str_wrk = string[:ind_hld]
        tm_strt_wrk, tm_end_wrk = fnd_tm_strt_end(str_wrk)
        if 'выходны' in string:
            str_hld = string[ind_hld:]
            tm_strt_hld, tm_end_hld = fnd_tm_strt_end(str_hld)
        else:
            tm_strt_hld, tm_end_hld = '-','-'
    #
    elif 'выходны' in string:
        if 'рабочи' in string:
            continue
        else:
            tm_strt_wrk, tm_end_wrk = '-','-'
            tm_strt_hld, tm_end_hld = fnd_tm_strt_end(string)
    #
    else:
        tm_strt_wrk, tm_end_wrk = fnd_tm_strt_end(string)
        tm_strt_hld, tm_end_hld = tm_strt_wrk, tm_end_wrk
    #
    def mk_tm(one):
        if one != '-':
            one = str(one) + ':00'
        return one
    # 
    lst_strt_wrk.append(mk_tm(tm_strt_wrk))
    lst_end_wrk.append(mk_tm(tm_end_wrk))
    lst_strt_hld.append(mk_tm(tm_strt_hld))
    lst_end_hld.append(mk_tm(tm_end_hld))
# 


In [120]:
# data1=list(zip(lst_reg_rab,lst_strt_wrk,lst_end_wrk,lst_strt_hld,lst_end_hld))
# columns1=['string','wrk_strt','wrk_end','hld_strt','hld_end']

# tmp_df = pd.DataFrame(data=data1,columns=columns1)

## Интервал

In [177]:
def fnd_interv(string):
    ind_sep = string.find(':')
    if ind_sep == -1:
        ind_sep = 0
    #
    ind_mnt = string.find('мин')
    str_part = string[ind_sep:ind_mnt]

    lst_dgt = re.findall(r'\d+', str_part)
    if len(lst_dgt) > 0:

        l1 = []
        for dgt in lst_dgt:
            l1.append(int(dgt))
        #
        l1.sort()
        min_interv = min(l1)
        max_interv = max(l1)
    #
    else:
        min_interv = 1
        max_interv = 180
    #

    return min_interv, max_interv

In [183]:
def fnd_day_tp(string,tp,rplc_min,replc_max):
    if tp in string:
        ind_tp = string.find(tp)
        if tp == 'мин':
            str_tp = string[:ind_tp]
        else:
            str_tp = string[ind_tp:]
        min_res,max_res = fnd_interv(str_tp)
    else:
        min_res,max_res = rplc_min,replc_max
    #
    return min_res,max_res

In [190]:
lst_interv = list(df.интервалы)

lst_wrk_min,lst_wrk_max,lst_hld_min,lst_hld_max = [],[],[],[]
lst_evn_wrk_min,lst_evn_wrk_max,lst_evn_hld_min,lst_evn_hld_max = [],[],[],[]

i=0
for i in range(len(lst_interv)):
    string=str(lst_interv[i])
    if ((string == 'nan') | ('расписани' in string)):
        wrk_min,wrk_max,hld_min,hld_max = '-','-','-','-'
        evn_wrk_min,evn_wrk_max,evn_hld_min,evn_hld_max = '-','-','-','-'
    elif (('рабочи' in string) | ('выходн' in string) | ('вечерн' in string)):
        wrk_min,wrk_max = fnd_day_tp(string,'рабочи',0,0)
        if ((wrk_min==0)&(wrk_max == 0)):
            wrk_min,wrk_max = fnd_day_tp(string,'мин',0,0)
        #
        hld_min,hld_max = fnd_day_tp(string,'выходн',wrk_min,wrk_max)
        evn_wrk_min,evn_wrk_max = fnd_day_tp(string,'вечерн',wrk_min,wrk_max)
    #
    elif 'мин' in string:
        wrk_min,wrk_max = fnd_interv(string)
        hld_min,hld_max,evn_wrk_min,evn_wrk_max = wrk_min,wrk_max,wrk_min,wrk_max
    else:
        wrk_min,wrk_max,hld_min,hld_max = 0,0,0,0
        evn_wrk_min,evn_wrk_max,evn_hld_min,evn_hld_max = 0,0,0,0
    #
    evn_hld_min,evn_hld_max = evn_wrk_min,evn_wrk_max
    
    lst_value=[wrk_min,wrk_max,hld_min,hld_max,
               evn_wrk_min,evn_wrk_max,evn_hld_min,evn_hld_max]
    lst_lsts=[lst_wrk_min,lst_wrk_max,lst_hld_min,lst_hld_max,
             lst_evn_wrk_min,lst_evn_wrk_max,lst_evn_hld_min,lst_evn_hld_max]
    j=0
    for j in range(len(lst_value)):
        lst_lsts[j].append(lst_value[j])
#

In [ ]:
# data2 = list(zip('wrk_min','wrk_max','hld_min','hld_max',
#                  'evn_wrk_min','evn_wrk_max','evn_hld_min','evn_hld_max'))

## Выпуск

In [254]:
def fnd_vpsk(string,tp):
    if tp == 'машин':
        ind_strt = 0
    elif tp == 'вагон':
        ind_strt = 0
    else:
        ind_strt = string.find(tp)
        if ind_strt == -1:
            ind_strt = 0
    #
    str_prt = string[ind_strt:]
    ind_veh = str_prt.find('машин')
    if ind_veh == -1:
        ind_veh = str_prt.find('вагон')
    str_tp = str_prt[:ind_veh]

    lst_dgt = re.findall(r'\d+', str_tp)
    if len(lst_dgt) > 0:
        l1 = []
        for dgt in lst_dgt:
            l1.append(int(dgt))
        #
        vpsk = max(l1)
    else:
        vpsk = 0
    #
    return vpsk

In [259]:
lst_vpsk = list(df.выпуск)

lst_vpsk_wrk,lst_vpsk_sbt,lst_vpsk_vsk = [],[],[]

i=0
for i in range(len(lst_vpsk)):
    string=str(lst_vpsk[i])
    if ((string == 'nan')):
        vpsk_wrk,vpsk_sbt,vpsk_vsk = '-','-','-'
        
    elif (('рабочи' in string) | ('выходн' in string) 
          | ('суббот' in string) | ('воскресен' in string)):
        vpsk_wrk = fnd_vpsk(string,'рабочим')
        vpsk_sbt = fnd_vpsk(string,'суббот')
        vpsk_vsk = fnd_vpsk(string,'воскресен')
        if ((vpsk_sbt == 0) | (vpsk_vsk == 0)):
            if vpsk_sbt == vpsk_vsk:
                vpsk_sbt = fnd_vpsk(string,'выходн')
                vpsk_vsk = vpsk_sbt
            else:
                if vpsk_vsk > 0:
                    vpsk_sbt = vpsk_vsk
                else:
                    vpsk_vsk = vpsk_sbt
        #
    #
    elif 'машин' in string:
        vpsk_wrk = fnd_vpsk(string,'машин')
        vpsk_sbt,vpsk_vsk = vpsk_wrk,vpsk_wrk
    elif 'вагон' in string:
        vpsk_wrk = fnd_vpsk(string,'вагон')
        vpsk_sbt,vpsk_vsk = vpsk_wrk,vpsk_wrk
    else:
        vpsk_wrk,vpsk_sbt,vpsk_vsk = 0,0,0
    #
    lst_vpsk_wrk.append(vpsk_wrk)
    lst_vpsk_sbt.append(vpsk_sbt)
    lst_vpsk_vsk.append(vpsk_vsk)
    
#

In [ ]:
# data3 = list(zip(lst_vpsk_wrk,lst_vpsk_sbt,lst_vpsk_vsk))

## Время в пути

In [ ]:
def fnd_dgt(string):
    if '66-го' in string:
        string = string.replace('66-го','')
    lst_dgt = re.findall(r'\d+', string)
    if len(lst_dgt) > 0:
        l1 = []
        for dgt in lst_dgt:
            l1.append(int(dgt))
        #
        tm_rt = max(l1)
    else:
        tm_rt = 0
    #
    return tm_rt

In [359]:
def get_hr_to_mnt(string):
    try:
        lst_splt = string.split('ч.')
        lst_dgt_hr = re.findall(r'\d+', lst_splt[0])
        hr = int(lst_dgt_hr[-1])

        lst_dgt_mnt = re.findall(r'\d+', lst_splt[1])
        mnt = int(lst_dgt_mnt[0])
        ttl_mnt = hr*60 + mnt
    except:
        
        try:
            lst_dgt_hr = re.findall(r'\d+', string)
            ttl_mnt = int(max(lst_dgt_hr))
        except:
            ttl_mnt = 0

    return ttl_mnt

In [368]:
lst_tm_rt_all = list(df.время_в_пути)

lst_tm_rt=[]

i=0
for i in range(len(lst_tm_rt_all)):
    string = str(lst_tm_rt_all[i])
    if string == 'nan':
        tm_rt = '-'
    #
    elif ('ч.' in string):
        string = string.replace('ч:','ч ')
        lst_hr = re.findall('мин', string)
        if len(lst_hr) == 1:
            tm_rt = get_hr_to_mnt(string)
        else:
            lst_prts=[]
            j=0
            for j in range(len(lst_hr)):
                ind_prt = string.find('мин')
                string1 = string[:ind_prt]
                string = string[ind_prt:].replace('мин','',1)
                try:
                    tm_rt1 = get_hr_to_mnt(string1)
                except:
                    print(i)
                    tm_rt1=0
                lst_prts.append(tm_rt1)
            # 
            tm_rt = max(lst_prts)

    elif 'мин' in string:
        tm_rt = fnd_dgt(string)
    else:
        tm_rt = 1111
    #
    lst_tm_rt.append(tm_rt)
# 

## Протяженность

In [390]:
lst_lngth_all = list(df.протяженность)

lst_lngth = []

i=0
for i in range(len(lst_lngth_all)):
    string = str(lst_lngth_all[i])
    if string == 'nan':
        lngth = '-'
    else:
        string = string.replace(',','.')
        lst_dgt_hr = re.findall("\d+\.\d+", string)
        if len(lst_dgt_hr) == 0:
            lst_dgt_hr = re.findall("\d+", string)
        #
        lst_dgt_flt=[]
        k=0
        for k in range(len(lst_dgt_hr)):
            dgt = float(lst_dgt_hr[k])
            lst_dgt_flt.append(dgt)
        #
        try:
            lngth = max(lst_dgt_flt)
        except:
            lngth='-'
    lst_lngth.append(lngth)
# 

## Подвижной состав

In [392]:
lst1 = list(df.подв_состав.unique())

## Стоимость проезда

In [416]:
lst_money_all = list(df.стоимость_проезда)

lst_money = []

i=0
for i in range(len(lst_money_all)):
    string = str(lst_money_all[i])
    if string == 'nan':
        money = '-'
    else:
        string = string.replace(',','.')
        lst_dgt_hr = re.findall("\d+", string)
        #
        lst_dgt_flt=[]
        k=0
        for k in range(len(lst_dgt_hr)):
            dgt = int(lst_dgt_hr[k])
            lst_dgt_flt.append(dgt)
        #
        try:
            lst_dgt_flt.sort()
            money = max(lst_dgt_flt)
            if money >= 1000:
                cnt=-1
                while money >= 1000:
                    money = max(lst_dgt_flt[:cnt])
                    cnt-=1
                #
        except:
            money='-'
    lst_money.append(money)
#